# Topic Modeling

In [7]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from nltk.corpus import stopwords
import gensim.corpora as corpora
from pprint import pprint
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from matplotlib import pyplot as plt

In [2]:
cali_df = pd.read_csv("cali_df.csv")
hk_df = pd.read_csv("reviews_hk.csv")
jap_df = pd.read_csv("reviews_jp.csv")

jap_df['name'] = 'japan'
cali_df['name'] = 'cali'
hk_df['name'] = 'hk'

def clean(df):
    df['rating'] = df['rating'].astype(str).str[0]
    df['rating'] = df['rating'].astype(int)
    date = df['date'].str.split(' ', n=1, expand = True)
    df['date'] = date[1]
    day = df['date']
    date = [datetime.strptime(x, '%B %d, %Y') for x in day]
    df['date'] = date
    return df

jap_df = clean(jap_df)
#cali_df = clean(cali_df)
hongk_df = clean(hk_df)

import re
def review_cleaning(text): 
    text = text.str.replace(r'[^a-z0-9A-Z_]', ' ', regex=True)
    text = text.str.lower()
    text = text.str.replace(r'\d+', '', regex=True)
    text = text.str.replace(r"\W", ' ', regex=True)
    text = text.str.replace(r"[^a-zA-Z0-9]+", ' ', regex=True)
    return text

jap_df['review'] = review_cleaning(jap_df['review'])
#cali_df['Review'] = review_cleaning(cali_df['Review'])
hk_df['review'] = review_cleaning(hk_df['review'])

hk_df.rename({"title":"Header", "review":"Review", "rating":"Rating", "date":"Date"}, axis=1, inplace=True)
jap_df.rename({"title":"Header", "review":"Review", "rating":"Rating", "date":"Date"}, axis=1, inplace=True)

jap_df['Header'] = review_cleaning(jap_df['Header'])
cali_df['Header'] = review_cleaning(cali_df['Header'])
hk_df['Header'] = review_cleaning(hk_df['Header'])

df_all = pd.concat([hk_df,jap_df,cali_df])
df_all = df_all.reset_index(drop=True)
df_all = df_all.drop_duplicates(subset=['Header','Review'], keep='first', inplace=False, ignore_index=False)
df_all["sentiment"] = np.where((df_all.Rating == 5) | (df_all.Rating == 4), "Positive","")
df_all["sentiment"] = np.where((df_all.Rating == 1) | (df_all.Rating == 2) | (df_all.Rating == 3), "Negative",df_all["sentiment"])
df_all['Date'] = pd.to_datetime(df_all['Date'])
df_all["Month"] = df_all['Date'].dt.month
df_all["Year"] = df_all['Date'].dt.year
df_all["Season"] = "Winter"
df_all["Season"] = np.where((df_all["Month"]==9)|(df_all["Month"]==10)|(df_all["Month"]==11), "Fall", df_all.Season)
df_all["Season"] = np.where((df_all["Month"]==6)|(df_all["Month"]==7)|(df_all["Month"]==8), "Summer", df_all.Season)
df_all["Season"] = np.where((df_all["Month"]==3)|(df_all["Month"]==4)|(df_all["Month"]==5), "Spring", df_all.Season)
df_all

,Header,Review,Rating,Date,name,sentiment,Month,Year,Season
0,a disney christmas indulge in disney s festive...,christmas is right around the corner and noth...,4,2021-12-30,hk,Positive,12,2021,Winter
1,disappointed,it wss christmas day but most of the outdoor s...,1,2021-12-27,hk,Negative,12,2021,Winter
2,disappointing terrible unbelievable even for r...,i guess there s no control on the limit of the...,1,2021-12-26,hk,Negative,12,2021,Winter
3,not a magical day,wifi and network are not stable ticketing and ...,2,2021-12-26,hk,Negative,12,2021,Winter
4,very long queuing time terrible experience,unacceptable queueing time not only for the ga...,1,2021-12-23,hk,Negative,12,2021,Winter
...,...,...,...,...,...,...,...,...,...
20874,like a big comfort blanket,i have just returned from a family holiday to ...,5,2017-04-21,cali,Positive,4,2017,Spring
20875,enchanting experience,the last time we visited disneyland was a few ...,5,2017-04-20,cali,Positive,4,2017,Spring
20876,a magical day at disneyland,this was our rd trip to disneyland and every t...,5,2017-04-20,cali,Positive,4,2017,Spring
20877,dance tour,disney was as magical as ever and did not disa...,5,2017-04-20,cali,Positive,4,2017,Spring


In [224]:
#!pip install gensim
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from gensim.models import Phrases
stop_words = stopwords.words('english')
stop_words.extend(['Disney', 'Disneyland','disney','cast','member','members','highly','recommended', 
                    're', 'edu', 'use','disneyland','hong','kong','japan','Los Angeles','Hong','Kong',
                    'Japan','los angeles','Los angeles','los angeles','universal','Universal','studio','Studio',
                    'birthday','ice','cream','years','year','halloween','Halloween','per','fast','pass','person',
                    'east','coast','make','sure','year','old','theme park','happiest','plan','expectations','formed',
                    'theme','Theme','park',"would","recommend", "even","though","well","better",
                    "day","back","come","main","california","mountain","first","felt","feel","angeles","china","looking",'forward',
                    #"story","king","star","jones","man"
                    ])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/taeholee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [132]:
#positvie
df_all_pos = df_all[df_all.Rating > 3]

#negative
df_all_neg = df_all[df_all.Rating <= 3]

In [146]:
alldata = df_all_pos.Header.values.tolist()
alldata_words = list(sent_to_words(alldata))

allbigram = gensim.models.Phrases(alldata_words, min_count=4, threshold=100
) # higher threshold fewer phrases.
allbigram_mod = gensim.models.phrases.Phraser(allbigram)

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
def jpmake_bigrams(texts):
    return [allbigram_mod[doc] for doc in texts]
alldata_words_nostops = remove_stopwords(alldata_words)
alldata_words_bigrams = jpmake_bigrams(alldata_words_nostops)

allbigram = []
allbigramlist = []
for i in range(len(alldata_words_bigrams)):
    allbigram=[]
    for j in range(len(alldata_words_bigrams[i])):
        if alldata_words_bigrams[i] not in stopwords.words("english"):
            if len(alldata_words_bigrams[i][j].split("_")) > 1:
                allbigram.append(alldata_words_bigrams[i][j])
    if len(allbigram) >= 1:
        allbigramlist.append(allbigram)

# Create Dictionary
allid2word = corpora.Dictionary(allbigramlist)
# Create Corpus
alltexts = allbigramlist
# Term Document Frequency
allcorpus = [allid2word.doc2bow(text) for text in alltexts]
# View

# number of topics
num_topics = 4
# Build LDA model
alllda_model = gensim.models.LdaMulticore(corpus=allcorpus,
                                       id2word=allid2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(alllda_model.print_topics())
alldoc_lda = alllda_model[allcorpus]

pyLDAvis.enable_notebook()
jpvis = gensimvis.prepare(alllda_model, allcorpus, allid2word)
jpvis

[(0,
  '0.365*"spring_break" + 0.133*"customer_service" + 0.133*"lion_king" + '
  '0.086*"fairy_tale" + 0.059*"wish_upon" + 0.037*"every_minute" + '
  '0.035*"gluten_free" + 0.035*"grown_ups" + 0.019*"star_wars" + '
  '0.014*"bucket_list"'),
 (1,
  '0.329*"star_wars" + 0.187*"spring_break" + 0.164*"bucket_list" + '
  '0.156*"galaxy_edge" + 0.088*"every_minute" + 0.016*"fairy_tale" + '
  '0.015*"customer_service" + 0.005*"toy_story" + 0.005*"never_disappoints" + '
  '0.005*"gluten_free"'),
 (2,
  '0.207*"th_anniversary" + 0.158*"fairy_tale" + 0.157*"comes_alive" + '
  '0.132*"nothing_beats" + 0.108*"grown_ups" + 0.059*"gluten_free" + '
  '0.035*"wish_upon" + 0.029*"spring_break" + 0.020*"star_wars" + '
  '0.016*"bucket_list"'),
 (3,
  '0.258*"never_disappoints" + 0.209*"pleasantly_surprised" + '
  '0.186*"toy_story" + 0.098*"gluten_free" + 0.052*"wish_upon" + '
  '0.046*"star_wars" + 0.032*"grown_ups" + 0.025*"spring_break" + '
  '0.013*"bucket_list" + 0.012*"fairy_tale"')]


/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.242818 -0.038063       1        1  31.772567
0      0.056331  0.066579       2        1  24.822901
3     -0.153157 -0.185470       3        1  21.715463
2     -0.145993  0.156953       4        1  21.689069, topic_info=                    Term       Freq      Total Category  logprob  loglift
1      never_disappoints  11.000000  11.000000  Default  17.0000  17.0000
0              star_wars  22.000000  22.000000  Default  16.0000  16.0000
4           spring_break  30.000000  30.000000  Default  15.0000  15.0000
14        th_anniversary   9.000000   9.000000  Default  14.0000  14.0000
10  pleasantly_surprised   9.000000   9.000000  Default  13.0000  13.0000
..                   ...        ...        ...      ...      ...      ...
9              toy_story   0.436643   8.847017   Topic4  -4.5260  -1.4804
16           galaxy_edge   0.496538  10.758932   Topic4  -4.3975  -1.5475
1      never_disappoints   0.500621  11.797293   Topic4  -4.3893  -1.6314
4           spring_break   1.162267  30.079828   Topic4  -3.5470  -1.7251
0              star_wars   0.797588  22.942822   Topic4  -3.9235  -1.8308

[85 rows x 6 columns], token_table=      Topic      Freq                  Term
term                                       
2         1  0.866852           bucket_list
2         2  0.086685           bucket_list
2         3  0.086685           bucket_list
2         4  0.086685           bucket_list
6         4  0.806567           comes_alive
8         1  0.128273      customer_service
8         2  0.769639      customer_service
5         1  0.634893          every_minute
5         2  0.253957          every_minute
5         4  0.126979          every_minute
11        1  0.084875            fairy_tale
11        2  0.339499            fairy_tale
11        4  0.509249            fairy_tale
16        1  0.836514           galaxy_edge
16        2  0.092946           galaxy_edge
13        2  0.241678           gluten_free
13        3  0.483357           gluten_free
13        4  0.241678           gluten_free
12        2  0.266842             grown_ups
12        3  0.133421             grown_ups
12        4  0.533684             grown_ups
7         2  0.845750             lion_king
1         2  0.084765     never_disappoints
1         3  0.847652     never_disappoints
1         4  0.084765     never_disappoints
3         4  0.785424         nothing_beats
10        2  0.101599  pleasantly_surprised
10        3  0.812792  pleasantly_surprised
10        4  0.101599  pleasantly_surprised
4         1  0.365694          spring_break
4         2  0.565163          spring_break
4         3  0.033245          spring_break
4         4  0.033245          spring_break
0         1  0.828146             star_wars
0         2  0.043587             star_wars
0         3  0.087173             star_wars
0         4  0.043587             star_wars
14        2  0.104223        th_anniversary
14        4  0.833781        th_anniversary
9         2  0.113032             toy_story
9         3  0.904260             toy_story
15        2  0.463132             wish_upon
15        3  0.308755             wish_upon
15        4  0.154377             wish_upon, R=17, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 4, 3])

In [157]:
alldata = df_all_neg.Header.values.tolist()
alldata_words = list(sent_to_words(alldata))

allbigram = gensim.models.Phrases(alldata_words, min_count=10, threshold=10
) # higher threshold fewer phrases.
allbigram_mod = gensim.models.phrases.Phraser(allbigram)

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
def jpmake_bigrams(texts):
    return [allbigram_mod[doc] for doc in texts]
alldata_words_nostops = remove_stopwords(alldata_words)
alldata_words_bigrams = jpmake_bigrams(alldata_words_nostops)

allbigram = []
allbigramlist = []
for i in range(len(alldata_words_bigrams)):
    allbigram=[]
    for j in range(len(alldata_words_bigrams[i])):
        if alldata_words_bigrams[i] not in stopwords.words("english"):
            if len(alldata_words_bigrams[i][j].split("_")) > 1:
                allbigram.append(alldata_words_bigrams[i][j])
    if len(allbigram) >= 1:
        allbigramlist.append(allbigram)

# Create Dictionary
allid2word = corpora.Dictionary(allbigramlist)
# Create Corpus
alltexts = allbigramlist
# Term Document Frequency
allcorpus = [allid2word.doc2bow(text) for text in alltexts]
# View

# number of topics
num_topics = 3
# Build LDA model
alllda_model = gensim.models.LdaMulticore(corpus=allcorpus,
                                       id2word=allid2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(alllda_model.print_topics())
alldoc_lda = alllda_model[allcorpus]

pyLDAvis.enable_notebook()
jpvis = gensimvis.prepare(alllda_model, allcorpus, allid2word)
jpvis

[(0,
  '0.540*"many_people" + 0.319*"long_lines" + 0.058*"customer_service" + '
  '0.055*"long_queues" + 0.028*"young_children"'),
 (1,
  '0.450*"long_queues" + 0.306*"young_children" + 0.106*"long_lines" + '
  '0.073*"many_people" + 0.066*"customer_service"'),
 (2,
  '0.573*"customer_service" + 0.213*"long_lines" + 0.088*"many_people" + '
  '0.065*"long_queues" + 0.061*"young_children"')]


/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.122186 -0.083533       1        1  41.056196
1     -0.158855 -0.036535       2        1  31.419631
2      0.036669  0.120068       3        1  27.524173, topic_info=               Term       Freq      Total Category  logprob  loglift
0  customer_service  20.000000  20.000000  Default   5.0000   5.0000
2       long_queues  18.000000  18.000000  Default   4.0000   4.0000
1       many_people  27.000000  27.000000  Default   3.0000   3.0000
4    young_children  12.000000  12.000000  Default   2.0000   2.0000
3        long_lines  22.000000  22.000000  Default   1.0000   1.0000
1       many_people  22.630779  27.431626   Topic1  -0.6154   0.6978
3        long_lines  13.347763  22.719320   Topic1  -1.1434   0.3584
2       long_queues   2.299390  18.522842   Topic1  -2.9021  -1.1961
0  customer_service   2.437589  20.660804   Topic1  -2.8437  -1.2470
4    young_children   1.161798  12.665407   Topic1  -3.5848  -1.4987
2       long_queues  14.410794  18.522842   Topic2  -0.7993   0.9067
4    young_children   9.796855  12.665407   Topic2  -1.1852   0.9009
3        long_lines   3.390472  22.719320   Topic2  -2.2463  -0.7445
0  customer_service   2.125210  20.660804   Topic2  -2.7134  -1.1166
1       many_people   2.324694  27.431626   Topic2  -2.6236  -1.3104
0  customer_service  16.098005  20.660804   Topic3  -0.5562   1.0406
3        long_lines   5.981086  22.719320   Topic3  -1.5463  -0.0445
4    young_children   1.706754  12.665407   Topic3  -2.8003  -0.7142
2       long_queues   1.812658  18.522842   Topic3  -2.7401  -1.0341
1       many_people   2.476153  27.431626   Topic3  -2.4282  -1.1149, token_table=      Topic      Freq              Term
term                                   
0         1  0.096802  customer_service
0         2  0.096802  customer_service
0         3  0.774413  customer_service
3         1  0.572200        long_lines
3         2  0.132046        long_lines
3         3  0.264092        long_lines
2         1  0.107975       long_queues
2         2  0.755824       long_queues
2         3  0.107975       long_queues
1         1  0.838448       many_people
1         2  0.072909       many_people
1         3  0.072909       many_people
4         1  0.078955    young_children
4         2  0.789552    young_children
4         3  0.157910    young_children, R=5, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])

In [178]:
alldata = df_all_pos.Review.values.tolist()
alldata_words = list(sent_to_words(alldata))

allbigram = gensim.models.Phrases(alldata_words, min_count=3, threshold=10
) # higher threshold fewer phrases.
allbigram_mod = gensim.models.phrases.Phraser(allbigram)

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
def jpmake_bigrams(texts):
    return [allbigram_mod[doc] for doc in texts]
alldata_words_nostops = remove_stopwords(alldata_words)
alldata_words_bigrams = jpmake_bigrams(alldata_words_nostops)

allbigram = []
allbigramlist = []
for i in range(len(alldata_words_bigrams)):
    allbigram=[]
    for j in range(len(alldata_words_bigrams[i])):
        if alldata_words_bigrams[i] not in stopwords.words("english"):
            if len(alldata_words_bigrams[i][j].split("_")) > 1:
                allbigram.append(alldata_words_bigrams[i][j])
    if len(allbigram) >= 1:
        allbigramlist.append(allbigram)

# Create Dictionary
allid2word = corpora.Dictionary(allbigramlist)
# Create Corpus
alltexts = allbigramlist
# Term Document Frequency
allcorpus = [allid2word.doc2bow(text) for text in alltexts]
# View

# number of topics
num_topics = 4
# Build LDA model
alllda_model = gensim.models.LdaMulticore(corpus=allcorpus,
                                       id2word=allid2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(alllda_model.print_topics())
alldoc_lda = alllda_model[allcorpus]

pyLDAvis.enable_notebook()
jpvis = gensimvis.prepare(alllda_model, allcorpus, allid2word)
jpvis

[(0,
  '0.028*"small_world" + 0.022*"mystic_manor" + 0.015*"lion_king" + '
  '0.013*"wait_times" + 0.012*"iron_man" + 0.011*"grizzly_gulch" + '
  '0.009*"young_children" + 0.009*"star_wars" + 0.009*"mickey_mouse" + '
  '0.008*"toy_story"'),
 (1,
  '0.027*"toy_story" + 0.013*"long_queue" + 0.010*"wait_times" + '
  '0.009*"really_enjoyed" + 0.008*"indiana_jones" + 0.008*"night_parade" + '
  '0.007*"star_wars" + 0.007*"adventure_land" + 0.007*"haunted_mansion" + '
  '0.006*"buy_tickets"'),
 (2,
  '0.018*"small_world" + 0.016*"two_days" + 0.013*"many_people" + '
  '0.013*"mickey_minnie" + 0.013*"popular_rides" + 0.012*"must_see" + '
  '0.011*"much_smaller" + 0.010*"haunted_mansion" + 0.010*"lion_king" + '
  '0.008*"magic_kingdom"'),
 (3,
  '0.030*"lion_king" + 0.015*"night_parade" + 0.014*"star_wars" + '
  '0.013*"roller_coaster" + 0.011*"mickey_mouse" + 0.011*"long_lines" + '
  '0.010*"grizzly_gulch" + 0.010*"toy_story" + 0.009*"long_queues" + '
  '0.007*"young_kids"')]


/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.001243  0.038332       1        1  27.070268
0     -0.007606  0.042962       2        1  26.942677
2     -0.055851 -0.046736       3        1  24.342356
1      0.064700 -0.034558       4        1  21.644699, topic_info=                 Term        Freq       Total Category  logprob  loglift
17          toy_story  709.000000  709.000000  Default  30.0000  30.0000
58        small_world  860.000000  860.000000  Default  29.0000  29.0000
11         long_queue  242.000000  242.000000  Default  28.0000  28.0000
31          lion_king  935.000000  935.000000  Default  27.0000  27.0000
46        many_people  277.000000  277.000000  Default  26.0000  26.0000
..                ...         ...         ...      ...      ...      ...
216  definitely_worth   50.724814  198.265346   Topic4  -5.5459   0.1672
58        small_world   73.981416  860.065392   Topic4  -5.1685  -0.9228
355       long_queues   54.867214  298.215242   Topic4  -5.4674  -0.1625
31          lion_king   59.933282  935.503675   Topic4  -5.3790  -1.2174
299         around_pm   47.307874  217.420915   Topic4  -5.6156   0.0053

[342 rows x 6 columns], token_table=      Topic      Freq              Term
term                                   
18        1  0.635054  absolutely_loved
18        2  0.233967  absolutely_loved
18        3  0.044565  absolutely_loved
18        4  0.077989  absolutely_loved
535       1  0.149024  admission_ticket
...     ...       ...               ...
246       4  0.074115        young_kids
503       1  0.209713  younger_children
503       2  0.039945  younger_children
503       3  0.149795  younger_children
503       4  0.599179  younger_children

[785 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 3, 2])

In [228]:
alldata = df_all_neg.Review.values.tolist()
alldata_words = list(sent_to_words(alldata))

allbigram = gensim.models.Phrases(alldata_words, min_count=20, threshold=300
) # higher threshold fewer phrases.
allbigram_mod = gensim.models.phrases.Phraser(allbigram)

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
def jpmake_bigrams(texts):
    return [allbigram_mod[doc] for doc in texts]
alldata_words_nostops = remove_stopwords(alldata_words)
alldata_words_bigrams = jpmake_bigrams(alldata_words_nostops)

allbigram = []
allbigramlist = []
for i in range(len(alldata_words_bigrams)):
    allbigram=[]
    for j in range(len(alldata_words_bigrams[i])):
        if alldata_words_bigrams[i] not in stopwords.words("english"):
            if len(alldata_words_bigrams[i][j].split("_")) > 1:
                allbigram.append(alldata_words_bigrams[i][j])
    if len(allbigram) >= 1:
        allbigramlist.append(allbigram)

# Create Dictionary
allid2word = corpora.Dictionary(allbigramlist)
# Create Corpus
alltexts = allbigramlist
# Term Document Frequency
allcorpus = [allid2word.doc2bow(text) for text in alltexts]
# View

# number of topics
num_topics = 3
# Build LDA model
alllda_model = gensim.models.LdaMulticore(corpus=allcorpus,
                                       id2word=allid2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(alllda_model.print_topics())
alldoc_lda = alllda_model[allcorpus]

pyLDAvis.enable_notebook()
jpvis = gensimvis.prepare(alllda_model, allcorpus, allid2word)
jpvis

[(0,
  '0.166*"lion_king" + 0.147*"indiana_jones" + 0.144*"haunted_mansion" + '
  '0.058*"roller_coaster" + 0.057*"peter_pan" + 0.056*"jungle_cruise" + '
  '0.051*"mystic_manor" + 0.051*"iron_man" + 0.047*"star_wars" + '
  '0.045*"spring_break"'),
 (1,
  '0.245*"star_wars" + 0.195*"toy_story" + 0.066*"lion_king" + '
  '0.052*"river_cruise" + 0.051*"single_rider" + 0.044*"haunted_mansion" + '
  '0.042*"mystic_manor" + 0.040*"star_tours" + 0.038*"sleeping_beauty" + '
  '0.032*"bucket_list"'),
 (2,
  '0.158*"roller_coaster" + 0.111*"buzz_lightyear" + 0.105*"grizzly_gulch" + '
  '0.102*"toy_story" + 0.066*"selfie_stick" + 0.065*"sunny_bay" + '
  '0.050*"roller_coasters" + 0.048*"jungle_cruise" + 0.044*"lion_king" + '
  '0.039*"mystic_manor"')]


/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.022987 -0.110944       1        1  38.321930
1     -0.112098  0.036460       2        1  33.899192
2      0.089111  0.074484       3        1  27.778879, topic_info=               Term        Freq       Total Category  logprob  loglift
8         star_wars  139.000000  139.000000  Default  22.0000  22.0000
0         toy_story  127.000000  127.000000  Default  21.0000  21.0000
6    roller_coaster   90.000000   90.000000  Default  20.0000  20.0000
13    grizzly_gulch   52.000000   52.000000  Default  19.0000  19.0000
3     indiana_jones   94.000000   94.000000  Default  18.0000  18.0000
..              ...         ...         ...      ...      ...      ...
3     indiana_jones    8.046304   94.398243   Topic3  -3.8086  -1.1814
7   sleeping_beauty    1.892598   22.344395   Topic3  -5.2559  -1.1877
4         peter_pan    3.388543   40.383686   Topic3  -4.6734  -1.1971
2   haunted_mansion    5.751524   97.401395   Topic3  -4.1444  -1.5485
8         star_wars    7.631621  139.305759   Topic3  -3.8615  -1.6235

[88 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
20        1  0.074889     bucket_list
20        2  0.524220     bucket_list
20        3  0.374443     bucket_list
17        1  0.279183  buzz_lightyear
17        2  0.117551  buzz_lightyear
...     ...       ...             ...
12        2  0.123141       sunny_bay
12        3  0.738845       sunny_bay
0         1  0.039199       toy_story
0         2  0.674223       toy_story
0         3  0.290073       toy_story

[66 rows x 3 columns], R=22, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])

In [186]:
alldata = df_all_neg[df_all_neg.name=="cali"].Review.values.tolist()
alldata_words = list(sent_to_words(alldata))

allbigram = gensim.models.Phrases(alldata_words, min_count=10, threshold=100
) # higher threshold fewer phrases.
allbigram_mod = gensim.models.phrases.Phraser(allbigram)

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
def jpmake_bigrams(texts):
    return [allbigram_mod[doc] for doc in texts]
alldata_words_nostops = remove_stopwords(alldata_words)
alldata_words_bigrams = jpmake_bigrams(alldata_words_nostops)

allbigram = []
allbigramlist = []
for i in range(len(alldata_words_bigrams)):
    allbigram=[]
    for j in range(len(alldata_words_bigrams[i])):
        if alldata_words_bigrams[i] not in stopwords.words("english"):
            if len(alldata_words_bigrams[i][j].split("_")) > 1:
                allbigram.append(alldata_words_bigrams[i][j])
    if len(allbigram) >= 1:
        allbigramlist.append(allbigram)

# Create Dictionary
allid2word = corpora.Dictionary(allbigramlist)
# Create Corpus
alltexts = allbigramlist
# Term Document Frequency
allcorpus = [allid2word.doc2bow(text) for text in alltexts]
# View

# number of topics
num_topics = 4
# Build LDA model
alllda_model = gensim.models.LdaMulticore(corpus=allcorpus,
                                       id2word=allid2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(alllda_model.print_topics())
alldoc_lda = alllda_model[allcorpus]

pyLDAvis.enable_notebook()
jpvis = gensimvis.prepare(alllda_model, allcorpus, allid2word)
jpvis

[(0,
  '0.367*"star_wars" + 0.085*"parking_garage" + 0.072*"six_flags" + '
  '0.068*"peter_pan" + 0.065*"anytime_soon" + 0.061*"galaxy_edge" + '
  '0.045*"haunted_mansion" + 0.041*"mid_week" + 0.023*"magic_kingdom" + '
  '0.019*"customer_service"'),
 (1,
  '0.280*"indiana_jones" + 0.092*"haunted_mansion" + 0.073*"magic_kingdom" + '
  '0.060*"city_hall" + 0.056*"peter_pan" + 0.046*"star_tours" + '
  '0.043*"haunted_house" + 0.039*"crowd_control" + 0.033*"spring_break" + '
  '0.033*"star_wars"'),
 (2,
  '0.158*"haunted_mansion" + 0.113*"jungle_cruise" + 0.087*"berry_farm" + '
  '0.084*"blue_bayou" + 0.083*"guest_services" + 0.076*"customer_service" + '
  '0.047*"minnie_mouse" + 0.039*"galaxy_edge" + 0.039*"mid_week" + '
  '0.033*"crowd_control"'),
 (3,
  '0.222*"customer_service" + 0.179*"star_wars" + 0.087*"spring_break" + '
  '0.057*"single_rider" + 0.056*"bucket_list" + 0.050*"selfie_stick" + '
  '0.046*"indiana_jones" + 0.041*"roller_coaster" + 0.040*"haunted_mansion" + '
  '0.034*"l

/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.115576 -0.076286       1        1  32.267784
0      0.156635  0.072142       2        1  24.239984
3      0.078010 -0.099571       3        1  23.278411
2     -0.119069  0.103715       4        1  20.213821, topic_info=                 Term        Freq       Total Category  logprob  loglift
3           star_wars  113.000000  113.000000  Default  30.0000  30.0000
8       indiana_jones   84.000000   84.000000  Default  29.0000  29.0000
6    customer_service   57.000000   57.000000  Default  28.0000  28.0000
7       jungle_cruise   24.000000   24.000000  Default  27.0000  27.0000
24     guest_services   16.000000   16.000000  Default  26.0000  26.0000
..                ...         ...         ...      ...      ...      ...
3           star_wars    4.060871  113.203882   Topic4  -3.6501  -1.7290
17  parking_structure    0.423544   12.297017   Topic4  -5.9106  -1.7697
22          six_flags    0.504187   16.197877   Topic4  -5.7363  -1.8709
9        single_rider    0.488458   16.846194   Topic4  -5.7680  -1.9418
1           peter_pan    0.643008   27.705351   Topic4  -5.4931  -2.1644

[150 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
23        2  0.699721  anytime_soon
23        3  0.116620  anytime_soon
23        4  0.174930  anytime_soon
16        1  0.055502    berry_farm
16        2  0.111005    berry_farm
...     ...       ...           ...
3         3  0.282676     star_wars
3         4  0.035334     star_wars
12        1  0.458878     toy_story
12        3  0.152959     toy_story
12        4  0.305919     toy_story

[104 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 4, 3])

In [189]:
alldata = df_all_neg[df_all_neg.name=="japan"].Review.values.tolist()
alldata_words = list(sent_to_words(alldata))

allbigram = gensim.models.Phrases(alldata_words, min_count=4, threshold=100
) # higher threshold fewer phrases.
allbigram_mod = gensim.models.phrases.Phraser(allbigram)

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
def jpmake_bigrams(texts):
    return [allbigram_mod[doc] for doc in texts]
alldata_words_nostops = remove_stopwords(alldata_words)
alldata_words_bigrams = jpmake_bigrams(alldata_words_nostops)

allbigram = []
allbigramlist = []
for i in range(len(alldata_words_bigrams)):
    allbigram=[]
    for j in range(len(alldata_words_bigrams[i])):
        if alldata_words_bigrams[i] not in stopwords.words("english"):
            if len(alldata_words_bigrams[i][j].split("_")) > 1:
                allbigram.append(alldata_words_bigrams[i][j])
    if len(allbigram) >= 1:
        allbigramlist.append(allbigram)

# Create Dictionary
allid2word = corpora.Dictionary(allbigramlist)
# Create Corpus
alltexts = allbigramlist
# Term Document Frequency
allcorpus = [allid2word.doc2bow(text) for text in alltexts]
# View

# number of topics
num_topics = 4
# Build LDA model
alllda_model = gensim.models.LdaMulticore(corpus=allcorpus,
                                       id2word=allid2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(alllda_model.print_topics())
alldoc_lda = alllda_model[allcorpus]

pyLDAvis.enable_notebook()
jpvis = gensimvis.prepare(alllda_model, allcorpus, allid2word)
jpvis

[(0,
  '0.123*"speak_english" + 0.120*"monsters_inc" + 0.060*"next_door" + '
  '0.055*"big_thunder" + 0.047*"small_world" + 0.046*"customer_service" + '
  '0.045*"cinderella_castle" + 0.040*"school_holidays" + 0.038*"mid_week" + '
  '0.033*"haunted_mansion"'),
 (1,
  '0.114*"small_world" + 0.110*"haunted_mansion" + 0.091*"big_thunder" + '
  '0.067*"star_tours" + 0.052*"buzz_lightyear" + 0.041*"trip_advisor" + '
  '0.040*"train_station" + 0.040*"light_show" + 0.038*"haunted_house" + '
  '0.034*"looking_forward"'),
 (2,
  '0.108*"school_holidays" + 0.092*"young_children" + 0.060*"plus_side" + '
  '0.053*"pop_corn" + 0.052*"roller_coaster" + 0.051*"gift_shops" + '
  '0.051*"foreign_tourists" + 0.042*"bucket_list" + 0.041*"looking_forward" + '
  '0.038*"snow_white"'),
 (3,
  '0.156*"walking_around" + 0.103*"peak_season" + 0.097*"star_wars" + '
  '0.085*"mickey_mouse" + 0.071*"jungle_cruise" + 0.050*"single_rider" + '
  '0.047*"never_seen" + 0.037*"small_world" + 0.037*"star_tours" + '
  '0

/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.010563  0.098212       1        1  30.148071
0     -0.117825  0.068502       2        1  25.395332
2     -0.063338 -0.151643       3        1  22.863720
3      0.170600 -0.015071       4        1  21.592878, topic_info=               Term       Freq      Total Category  logprob  loglift
9    walking_around  19.000000  19.000000  Default  30.0000  30.0000
6     speak_english  17.000000  17.000000  Default  29.0000  29.0000
18      peak_season  12.000000  12.000000  Default  28.0000  28.0000
20     mickey_mouse   9.000000   9.000000  Default  27.0000  27.0000
23  school_holidays  17.000000  17.000000  Default  26.0000  26.0000
..              ...        ...        ...      ...      ...      ...
2   haunted_mansion   0.796581  19.440579   Topic4  -4.7814  -1.6620
6     speak_english   0.603457  17.355007   Topic4  -5.0590  -1.8262
29   young_children   0.427931  14.442488   Topic4  -5.4028  -1.9862
27        next_door   0.354492   8.753037   Topic4  -5.5910  -1.6737
4        light_show   0.337480   7.960141   Topic4  -5.6402  -1.6279

[163 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
0         1  0.567349     big_thunder
0         2  0.283675     big_thunder
0         4  0.141837     big_thunder
32        1  0.147847     bucket_list
32        3  0.591388     bucket_list
...     ...       ...             ...
9         3  0.104325  walking_around
9         4  0.782441  walking_around
29        1  0.207720  young_children
29        2  0.069240  young_children
29        3  0.623161  young_children

[119 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3, 4])

In [215]:
alldata = df_all_neg[df_all_neg.name=="hk"].Review.values.tolist()
alldata_words = list(sent_to_words(alldata))

allbigram = gensim.models.Phrases(alldata_words, min_count=15, threshold=200
) # higher threshold fewer phrases.
allbigram_mod = gensim.models.phrases.Phraser(allbigram)

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
def jpmake_bigrams(texts):
    return [allbigram_mod[doc] for doc in texts]
alldata_words_nostops = remove_stopwords(alldata_words)
alldata_words_bigrams = jpmake_bigrams(alldata_words_nostops)

allbigram = []
allbigramlist = []
for i in range(len(alldata_words_bigrams)):
    allbigram=[]
    for j in range(len(alldata_words_bigrams[i])):
        if alldata_words_bigrams[i] not in stopwords.words("english"):
            if len(alldata_words_bigrams[i][j].split("_")) > 1:
                allbigram.append(alldata_words_bigrams[i][j])
    if len(allbigram) >= 1:
        allbigramlist.append(allbigram)

# Create Dictionary
allid2word = corpora.Dictionary(allbigramlist)
# Create Corpus
alltexts = allbigramlist
# Term Document Frequency
allcorpus = [allid2word.doc2bow(text) for text in alltexts]
# View

# number of topics
num_topics = 4
# Build LDA model
alllda_model = gensim.models.LdaMulticore(corpus=allcorpus,
                                       id2word=allid2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(alllda_model.print_topics())
alldoc_lda = alllda_model[allcorpus]

pyLDAvis.enable_notebook()
jpvis = gensimvis.prepare(alllda_model, allcorpus, allid2word)
jpvis

[(0,
  '0.330*"lion_king" + 0.162*"roller_coaster" + 0.099*"jungle_river" + '
  '0.089*"buzz_lightyear" + 0.083*"mystic_manor" + 0.069*"iron_man" + '
  '0.047*"grizzly_gulch" + 0.040*"toy_story" + 0.031*"runaway_mine" + '
  '0.018*"star_wars"'),
 (1,
  '0.377*"toy_story" + 0.144*"lion_king" + 0.099*"mystic_manor" + '
  '0.063*"sleeping_beauty" + 0.057*"star_wars" + 0.037*"iron_man" + '
  '0.035*"jungle_river" + 0.035*"runaway_mine" + 0.034*"roller_coasters" + '
  '0.026*"roller_coaster"'),
 (2,
  '0.288*"lion_king" + 0.201*"iron_man" + 0.149*"buzz_lightyear" + '
  '0.070*"grizzly_gulch" + 0.068*"haunted_mansion" + 0.054*"roller_coasters" + '
  '0.047*"star_wars" + 0.032*"mystic_manor" + 0.028*"runaway_mine" + '
  '0.019*"roller_coaster"'),
 (3,
  '0.240*"roller_coaster" + 0.173*"grizzly_gulch" + 0.163*"sunny_bay" + '
  '0.143*"mystic_manor" + 0.066*"lion_king" + 0.064*"haunted_mansion" + '
  '0.035*"jungle_river" + 0.028*"roller_coasters" + 0.027*"toy_story" + '
  '0.014*"buzz_lightyea

/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.054027  0.133921       1        1  29.060017
3      0.156071 -0.008559       2        1  24.081385
2     -0.099233 -0.085238       3        1  23.551418
0     -0.002811 -0.040124       4        1  23.307180, topic_info=               Term        Freq       Total Category  logprob  loglift
1         toy_story   85.000000   85.000000  Default  15.0000  15.0000
5    roller_coaster   70.000000   70.000000  Default  14.0000  14.0000
2         lion_king  133.000000  133.000000  Default  13.0000  13.0000
10        sunny_bay   32.000000   32.000000  Default  12.0000  12.0000
4          iron_man   51.000000   51.000000  Default  11.0000  11.0000
..              ...         ...         ...      ...      ...      ...
1         toy_story    6.114856   85.085342   Topic4  -3.2236  -1.1765
3   haunted_mansion    1.732653   25.669073   Topic4  -4.4847  -1.2392
10        sunny_bay    1.783452   32.737853   Topic4  -4.4558  -1.4536
6   sleeping_beauty    0.556408   16.105938   Topic4  -5.6206  -1.9090
9   roller_coasters    0.547665   19.963851   Topic4  -5.6364  -2.1396

[75 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
13        1  0.071943   buzz_lightyear
13        2  0.047962   buzz_lightyear
13        3  0.551564   buzz_lightyear
13        4  0.335735   buzz_lightyear
11        1  0.099166    grizzly_gulch
11        2  0.555330    grizzly_gulch
11        3  0.218165    grizzly_gulch
11        4  0.138832    grizzly_gulch
3         1  0.116872  haunted_mansion
3         2  0.389574  haunted_mansion
3         3  0.389574  haunted_mansion
3         4  0.077915  haunted_mansion
4         1  0.136892         iron_man
4         2  0.039112         iron_man
4         3  0.606234         iron_man
4         4  0.215115         iron_man
12        1  0.248766     jungle_river
12        2  0.213228     jungle_river
12        3  0.035538     jungle_river
12        4  0.533071     jungle_river
2         1  0.209555        lion_king
2         2  0.082325        lion_king
2         3  0.336784        lion_king
2         4  0.381689        lion_king
8         1  0.319908     mystic_manor
8         2  0.387258     mystic_manor
8         3  0.084186     mystic_manor
8         4  0.218885     mystic_manor
14        1  0.831786     river_cruise
5         1  0.070447   roller_coaster
5         2  0.535397   roller_coaster
5         3  0.042268   roller_coaster
5         4  0.352235   roller_coaster
9         1  0.350634  roller_coasters
9         2  0.250453  roller_coasters
9         3  0.400724  roller_coasters
9         4  0.050091  roller_coasters
0         1  0.398340     runaway_mine
0         2  0.113811     runaway_mine
0         3  0.227623     runaway_mine
0         4  0.284529     runaway_mine
6         1  0.745067  sleeping_beauty
6         2  0.124178  sleeping_beauty
6         3  0.124178  sleeping_beauty
6         4  0.062089  sleeping_beauty
7         1  0.507435        star_wars
7         2  0.046130        star_wars
7         3  0.322913        star_wars
7         4  0.138391        star_wars
10        1  0.091637        sunny_bay
10        2  0.794188        sunny_bay
10        3  0.061091        sunny_bay
10        4  0.061091        sunny_bay
1         1  0.846209        toy_story
1         2  0.047012        toy_story
1         3  0.035259        toy_story
1         4  0.070517        toy_story, R=15, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 3, 1])